In [2]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [9]:
df_model = pd.read_pickle('../../data/final/NotIot_clean_EN_news_lexic.p')
df_model

,count_words,count_capital_words,count_excl_quest_marks,sentiment,origin_text,clean_text,lex_anger,lex_sadness,lex_joy,lex_fear
0,8,0,0,joy,Test to predict breast cancer relapse is approved,test predict breast cancer relapse approved,0.096167,0.234167,0.081000,0.224000
1,8,0,0,sadness,"Two Hussein allies are hanged, Iraqi official says",two hussein allies hanged iraqi official says,0.000000,0.000000,0.000000,0.000000
2,5,1,0,joy,Sights and sounds from CES,sights sounds ces,0.000000,0.000000,0.000000,0.000000
3,6,0,0,joy,Schuey sees Ferrari unveil new car,schuey sees ferrari unveil new car,0.000000,0.000000,0.000000,0.000000
4,8,0,0,fear,Closings and cancellations top advice on flu outbreak,closings cancellations top advice flu outbreak,0.000000,0.073000,0.000000,0.174500
5,5,0,0,sadness,Trucks swallowed in subway collapse,trucks swallowed subway collapse,0.000000,0.117250,0.000000,0.175750
6,6,0,0,surprise,Sarkozy letter surprises French cartoons hearing,sarkozy letter surprises french cartoons hearing,0.000000,0.000000,0.101000,0.026000
7,11,0,0,sadness,"Building a memorial to a son, one child at a time",building memorial son one child time,0.000000,0.091167,0.077667,0.000000
8,9,0,0,surprise,Lawmaker seeks iPod ban in crosswalks in New York,lawmaker seeks ipod ban crosswalks new york,0.000000,0.000000,0.000000,0.000000
9,5,0,0,sadness,Diabetic waits months for eyeglasses,diabetic waits months eyeglasses,0.000000,0.000000,0.000000,0.000000


In [ ]:
#df_model.head()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [11]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [12]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_excl_quest_marks','count_words']
    
    lexiccols = ['lex_anger','lex_joy','lex_sadness','lex_fear']

    features = FeatureUnion([ ('textcounts', ColumnExtractor(cols=textcountscols)),
                            ('lexiccols', ColumnExtractor(cols=lexiccols)),
                             ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

  
    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [13]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [14]:
mnb = MultinomialNB()

In [15]:
countvect = CountVectorizer()

In [16]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   30.7s finished


done in 31.718s

Best CV score: 0.524
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.404


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        14
        fear       0.37      0.47      0.41        15
         joy       0.46      0.74      0.57        35
     sadness       0.31      0.26      0.29        19
    surprise       0.25      0.12      0.17        16

   micro avg       0.40      0.40      0.40        99
   macro avg       0.28      0.32      0.29        99
weighted avg       0.32      0.40      0.35        99

all results
0.518059 (0.024445) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.523702 (0.013608) with: {'clf__alpha': 0.25, 'features__pipe__vect__ma

In [17]:
tfidfvect = TfidfVectorizer()

In [18]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   23.3s finished


done in 23.810s

Best CV score: 0.505
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.424


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        14
        fear       0.50      0.33      0.40        15
         joy       0.43      0.83      0.56        35
     sadness       0.47      0.37      0.41        19
    surprise       0.17      0.06      0.09        16

   micro avg       0.42      0.42      0.42        99
   macro avg       0.31      0.32      0.29        99
weighted avg       0.34      0.42      0.35        99

all results
0.504515 (0.014749) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.458239 (0.018959) with: {'clf__alpha': 0.25, 'features__pipe__vect__ma

In [14]:
#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 1)
#clf__alpha: 0.75
""""textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.25, min_df=1, ngram_range=(1,1)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.25))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)"""""

'"textcountscols = [\'count_capital_words\',\'count_emojis\',\'count_excl_quest_marks\',\'count_hashtags\'\n                      ,\'count_mentions\',\'count_urls\',\'count_words\']\n    \nfeatures = FeatureUnion([(\'textcounts\', ColumnExtractor(cols=textcountscols))\n                         , (\'pipe\', Pipeline([(\'cleantext\', ColumnExtractor(cols=\'clean_text\'))\n                                              , (\'vect\', CountVectorizer(max_df=0.25, min_df=1, ngram_range=(1,1)))]))]\n                       , n_jobs=-1)\n\npipeline = Pipeline([\n    (\'features\', features)\n    , (\'clf\', MultinomialNB(alpha=0.25))\n])\n\nbest_model = pipeline.fit(df_model.drop(\'sentiment\', axis=1), df_model.sentiment)'

In [15]:
#df_model_pos = pd.read_pickle('../data/df_model_pos_fr.p')
#best_model.predict(df_model_pos).tolist()

In [16]:
#df_model_neg = pd.read_pickle('../data/df_model_neg_fr.p')
#best_model.predict(df_model_neg).tolist()